In [1]:
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing

plt.gcf().clear()
plt.clf()
plt.cla()
plt.close()

In [2]:
def readfile(fname):
    file = open("Events/" + fname)
    events = []
    min_x = 0
    min_y = 0
    max_x = 0
    max_y = 0
    for line in file:
        data = line.split(' ')
        data[0] = int(data[0])
        if data[0] < min_x:
            min_x = data[0]
        if data[0] > max_x:
            max_x = data[0]
        data[1] = int(data[1])
        if data[1] < min_y:
            min_y = data[1]
        if data[1] > max_y:
            max_y = data[1]
        data[2] = int(data[2])
        data[3] = int(data[3].replace('\n',''))
        events.append(data)      #x_pixel, y_pixel, time_interval(10^-6 s), Polarity
    file.close()
    return sorted(events,key=lambda x: x[2]), max_x, max_y, min_x, min_y

In [3]:
time_interval = 256
def process_video (fname,generation_interval):
    events, max_x, max_y, min_x, min_y = readfile(fname)
    fname = fname.replace(".txt","")
    folder = "generated_images" + str(generation_interval) + "_sampled"
    if not os.path.exists(folder + "/pos/" + fname):
        os.makedirs(folder + "/pos/" + fname)
    if not os.path.exists(folder + "/neg/" + fname):
        os.makedirs(folder + "/neg/" + fname)
    if not os.path.exists(folder + "/both/" + fname):
        os.makedirs(folder + "/both/" + fname)
        for start_entry in np.arange(0, len(events), generation_interval):
            frame_start = events[start_entry][2]
            pos = Image.new("L", (max_x+1-min_x, max_y+1-min_y),color=0)      # "1 for single bit"
            neg = Image.new("L", (max_x+1-min_x, max_y+1-min_y),color=0)      # "1 for single bit"
            both = Image.new("L", (max_x+1-min_x, max_y+1-min_y),color=0)      # "1 for single bit"
            for window in range(256):
                n = start_entry
                window_start = (255-window)*time_interval+frame_start
                window_end = (256-window)*time_interval+frame_start
                while n<len(events)-1 and events[n][2]<=window_end:
                    if events[n][2]>=window_start:
                        both.putpixel((events[n][0]-min_x,events[n][1]-min_y),window)
                        if events[n][3] == 1:
                            pos.putpixel((events[n][0]-min_x,events[n][1]-min_y),window)
                        else:
                            neg.putpixel((events[n][0]-min_x,events[n][1]-min_y),window)
                    n += 1
            pos = pos.transpose(Image.FLIP_TOP_BOTTOM)
            pos.save(folder + "/pos/" + fname + "/" + str(int(start_entry//generation_interval)) + ".png", "PNG")
            neg = neg.transpose(Image.FLIP_TOP_BOTTOM)
            neg.save(folder + "/neg/" + fname + "/" + str(int(start_entry//generation_interval)) + ".png", "PNG")
            both = both.transpose(Image.FLIP_TOP_BOTTOM)
            both.save(folder + "/both/" + fname + "/" + str(int(start_entry//generation_interval)) + ".png", "PNG")

In [4]:
process_video ("1.txt",512)

In [ ]:
for fname in os.listdir("Events"):
    process_video (fname,512)

In [ ]:
"""
aggregation_time = 2000
        
def process_video (fname,generation_interval,aggregation_time):
    events, max_x, max_y, min_x, min_y = readfile(fname)
    fname = fname.replace(".txt","")
    folder = "generated_images" + str(generation_interval) + "_" + str(aggregation_time)
    if not os.path.exists(folder + "/pos/" + fname):
        os.makedirs(folder + "/pos/" + fname)
    if not os.path.exists(folder + "/neg/" + fname):
        os.makedirs(folder + "/neg/" + fname)
    if not os.path.exists(folder + "/both/" + fname):
        os.makedirs(folder + "/both/" + fname)
    for i in range(len(events)):
        if i%generation_interval == 0:
            pos_events = 0
            neg_events = 0
            n = i
            frame_start = events[i][2]
            pos = Image.new("1", (max_x+1-min_x, max_y+1-min_y),color=0)      # "1 for single bit"
            neg = Image.new("1", (max_x+1-min_x, max_y+1-min_y),color=0)      # "1 for single bit"
            both = Image.new("1", (max_x+1-min_x, max_y+1-min_y),color=0)      # "1 for single bit"
            while events[n][2]-frame_start<aggregation_time and n<len(events)-1:
                both.putpixel((events[n][0]-min_x,events[n][1]-min_y),1)
                if events[n][3] == 1:
                    pos.putpixel((events[n][0]-min_x,events[n][1]-min_y),1)
                    pos_events += 1
                else:
                    neg.putpixel((events[n][0]-min_x,events[n][1]-min_y),1)
                    neg_events += 1
                n += 1
            pos = pos.transpose(Image.FLIP_TOP_BOTTOM)
            pos.save(folder + "/pos/" + fname + "/" + str(int(i//generation_interval)) + ".png", "PNG")
            neg = neg.transpose(Image.FLIP_TOP_BOTTOM)
            neg.save(folder + "/neg/" + fname + "/" + str(int(i//generation_interval)) + ".png", "PNG")
            both = both.transpose(Image.FLIP_TOP_BOTTOM)
            both.save(folder + "/both/" + fname + "/" + str(int(i//generation_interval)) + ".png", "PNG")

In [6]:
max_x = 0
max_y = 0
max_frames = 0
min_x = float('Inf')
min_y = float('Inf')
min_frames = float('Inf')

for fname in os.listdir("E:/Jupyter/Project/generated_images512_timeSampled/both"):
    if os.listdir("E:/Jupyter/Project/generated_images512_timeSampled/both/" + fname):
        max_frame = 0
        for frame in os.listdir("E:/Jupyter/Project/generated_images512_timeSampled/both/" + fname):
            if int(frame.replace(".png","")) > max_frame:
                max_frame = int(frame.replace(".png",""))
        if max_frame > max_frames:
            max_frames = max_frame
        if max_frame < min_frames:
            min_frames = max_frame
        width, height = Image.open("E:/Jupyter/Project/generated_images512_timeSampled/both/" + fname + "/" + str(max_frame) + ".png").size
        if width > max_x:
            max_x = width
        if height > max_y:
            max_y = height
        if width < min_x:
            min_x = width
        if height < min_y:
            min_y = height
print(max_x,max_y,max_frames)
text_file = open("Output.txt", "w")
text_file.write("max_x: %d\n" % max_x)
text_file.write("max_y: %d\n" % max_y)
text_file.write("max_frames: %d\n" % max_frames)
text_file.write("min_x: %d\n" % min_x)
text_file.write("min_y: %d\n" % min_y)
text_file.write("min_frames: %d\n" % min_frames)
text_file.close()

176 100 663


In [6]:
from PIL import Image
import os
import numpy as np
import multiprocessing

def readfile(fname):
    file = open("events/" + fname)
    events = []
    for line in file:
        data = line.split(' ')
        data[0] = int(data[0])
        data[1] = int(data[1])
        data[2] = int(data[2])
        data[3] = int(data[3].replace('\n',''))
        events.append(data)      #x_pixel, y_pixel, time_interval, Polarity
    file.close()
    return sorted(events,key=lambda x: x[2])

time_interval = 1024
def process_video (fname):
    events = readfile(fname)
    fname = fname.replace(".txt","")
    folder = "D:\Project\eventbins/generated_images_timeSampled"
    if not os.path.exists(folder + "/pos/" + fname):
        os.makedirs(folder + "/pos/" + fname)
    if not os.path.exists(folder + "/neg/" + fname):
        os.makedirs(folder + "/neg/" + fname)
    frame_num = 0
    generate_interval =  len(events)//64 +1
    if generate_interval < 1024:
        generate_interval = 1024
    for start_entry in np.arange(generate_interval, len(events), generate_interval):
        frame_start = events[start_entry][2] #time of start entry
        pos = Image.new("L", (176, 100),color=0)      # "1 for single bit"
        neg = Image.new("L", (176, 100),color=0)      # "1 for single bit"
        curr_entry = start_entry
        while curr_entry>0 and events[curr_entry+1][2]>frame_start-time_interval*255:
            curr_entry -=1
        while curr_entry<start_entry:
            if events[curr_entry][3] == 1:
                pos.putpixel((events[curr_entry][0],events[curr_entry][1]),(frame_start-events[curr_entry][2])//time_interval)
            else:
                neg.putpixel((events[curr_entry][0],events[curr_entry][1]),(frame_start-events[curr_entry][2])//time_interval)
            curr_entry += 1
        pos = pos.transpose(Image.FLIP_TOP_BOTTOM)
        pos.save(folder + "/pos/" + fname + "/" + str(int(frame_num)) + ".png", "PNG")
        neg = neg.transpose(Image.FLIP_TOP_BOTTOM)
        neg.save(folder + "/neg/" + fname + "/" + str(int(frame_num)) + ".png", "PNG")
        frame_num += 1
        
process_video("1.txt")